In [ ]:
#ANNOTATIONS

#Libraries used in example: (Not necessary to run TLS)
import numpy
import scipy
#Everest is a pipeline for detrending lightcurves:
import everest
#Graphing:
from astropy.stats import sigma_clip
import matplotlib.pyplot as plt
#Logging library is used for managing errors and finding bugs:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('matplotlib')
logger.setLevel(logging.CRITICAL)
###

####Detrending and graphing code:
from matplotlib import rcParams; rcParams["figure.dpi"] = 150
EPIC_id = 201367065  
#Data sorting:      
star = everest.Everest(EPIC_id)
t = numpy.delete(star.time, star.badmask)
y = numpy.delete(star.fcor, star.badmask)
t = numpy.array(t[~numpy.isnan(y)], dtype='float32')
y = numpy.array(y[~numpy.isnan(y)], dtype='float32')
#Detrending with scipy
trend = scipy.signal.medfilt(y, 25)
y_filt = y /trend
y_filt = sigma_clip(y_filt, sigma_upper=2, sigma_lower=float('inf'))
#Plotting code:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6))
ax = axes[0]
#"t" is x-axis
ax.plot(t, y, "k")
ax.plot(t, trend)
ax.set_ylabel("Flux (electrons per sec)")
ax = axes[1]
ax.plot(t, y_filt, "k")
ax.set_xlim(t.min(), t.max())
ax.set_xlabel("Time (days)")
ax.set_ylabel("Normalized flux");
#################################################COPY THIS CODE####################################################################
#TRANSITLEAST SQUARES CODE:
from transitleastsquares import transitleastsquares
#Most import line: (to run tls its simply a function "transitleastsquares()" it takes x-axis and y-axis inputs--detrended.)
model = transitleastsquares(t, y_filt)
#not exactly sure what this is but I believe we could use the same arguments...
results = model.power(oversampling_factor=5, duration_grid_step=1.02)
###################################################################################################################################

In [ ]:
#plotting period graph
plt.figure()
ax = plt.gca()
#referencing results for plotting...
ax.axvline(results.period, alpha=0.4, lw=3)
plt.xlim(numpy.min(results.periods), numpy.max(results.periods))
for n in range(2, 10):
    ax.axvline(n*results.period, alpha=0.4, lw=1, linestyle="dashed")
    ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed")
plt.ylabel(r'SDE')
plt.xlabel('Period (days)')
plt.plot(results.periods, results.power, color='black', lw=0.5)
plt.xlim(0, max(results.periods));



In [ ]:
#Irrelevant code:
plt.figure()
plt.plot(
    results.model_folded_phase,
    results.model_folded_model,
    color='red')
#Again, using results for x and y arguments on scatter plot. If we are making a scatter plot we can use folded_phase and folded_y...
plt.scatter(
    results.folded_phase,
    results.folded_y,
#More irrelevant code:
    color='blue',
    s=10,
    alpha=0.5,
    zorder=2)
plt.xlim(0.49, 0.51)
plt.xlabel('Phase')
plt.ylabel('Relative flux');